In [58]:
import pandas as pd
from pathlib import Path

In [59]:
pd.set_option('display.max_columns', None)

In [60]:
gen_path = "../data/general_payments/cms_general_2024.csv"
res_path = "../data/research_payments/cms_research_2024.csv"

gen_usecols = [
    "Covered_Recipient_Profile_ID",
    "Covered_Recipient_NPI",
    "Covered_Recipient_First_Name",
    "Covered_Recipient_Last_Name",

    "Recipient_City",
    "Recipient_State",

    "Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID",
    "Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name",
    "Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_1",

    "Total_Amount_of_Payment_USDollars",
    "Date_of_Payment",
    "Nature_of_Payment_or_Transfer_of_Value", # used strictly for filtering and dropped later
    "Record_ID",
    "Program_Year",
]

res_usecols = [
    "Covered_Recipient_Profile_ID",
    "Covered_Recipient_NPI",
    "Covered_Recipient_First_Name",
    "Covered_Recipient_Last_Name",

    "Recipient_City",
    "Recipient_State",

    "Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID",
    "Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name",
    "Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_1",

    "Total_Amount_of_Payment_USDollars",
    "Date_of_Payment",
    "Record_ID",
    "Program_Year",
]

gen_df = pd.read_csv(gen_path, usecols=gen_usecols, low_memory=False)
gen_df = gen_df.reindex(columns=gen_usecols)

res_df = pd.read_csv(res_path, usecols=res_usecols, low_memory=False)
res_df = res_df.reindex(columns=res_usecols)

In [61]:
# CLEAN COLUMNS
print(gen_df.columns.tolist())
print(res_df.columns.tolist())

print([col.strip().lower() for col in gen_df.columns])
print([col.strip().lower() for col in res_df.columns])

gen_df.columns = [col.strip().lower() for col in gen_df.columns]
res_df.columns = [col.strip().lower() for col in res_df.columns]

['Covered_Recipient_Profile_ID', 'Covered_Recipient_NPI', 'Covered_Recipient_First_Name', 'Covered_Recipient_Last_Name', 'Recipient_City', 'Recipient_State', 'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID', 'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name', 'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_1', 'Total_Amount_of_Payment_USDollars', 'Date_of_Payment', 'Nature_of_Payment_or_Transfer_of_Value', 'Record_ID', 'Program_Year']
['Covered_Recipient_Profile_ID', 'Covered_Recipient_NPI', 'Covered_Recipient_First_Name', 'Covered_Recipient_Last_Name', 'Recipient_City', 'Recipient_State', 'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID', 'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name', 'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_1', 'Total_Amount_of_Payment_USDollars', 'Date_of_Payment', 'Record_ID', 'Program_Year']
['covered_recipient_profile_id', 'covered_recipient_npi', 'covered_recipient_first_name', 'cover

In [62]:
# FILTER UNRELATED PAYMENT NATURES SUCH AS FOOD AND DRINKS
useful_natures = [
    "Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program",
    "Consulting Fee",
    "Education",
    "Honoraria",
    "Royalty or License",
    "Compensation for serving as faculty or as a speaker for a medical education program",
    "Long term medical supply or device loan",
    "Grant",
]

print(len(gen_df))
gen_df = gen_df[gen_df['nature_of_payment_or_transfer_of_value'].isin(useful_natures)]
print(len(gen_df))

15397627
649560


In [63]:
# DROP NATURES USED FOR FILTERING TO CONCAT LATER
gen_df.drop(columns=["nature_of_payment_or_transfer_of_value"], inplace=True)
display(gen_df.head())

,covered_recipient_profile_id,covered_recipient_npi,covered_recipient_first_name,covered_recipient_last_name,recipient_city,recipient_state,applicable_manufacturer_or_applicable_gpo_making_payment_id,applicable_manufacturer_or_applicable_gpo_making_payment_name,name_of_drug_or_biological_or_device_or_medical_supply_1,total_amount_of_payment_usdollars,date_of_payment,record_id,program_year
0,NaN,NaN,NaN,NaN,New Orleans,LA,100000000226,"Genentech USA, Inc.",Ocrevus,10000.0,02/23/2024,1133195830,2024
1,NaN,NaN,NaN,NaN,Nashville,TN,100000000226,"Genentech USA, Inc.",Polivy,5000.0,08/01/2024,1133196475,2024
2,NaN,NaN,NaN,NaN,LA CROSSE,WI,100000686813,Linde Gas & Equipment Inc.,REQNOXBOX-I,330.0,02/13/2024,1142632901,2024
3,NaN,NaN,NaN,NaN,LA CROSSE,WI,100000686813,Linde Gas & Equipment Inc.,REQNOXBOX-I,330.0,08/06/2024,1142632912,2024
4,NaN,NaN,NaN,NaN,LA CROSSE,WI,100000686813,Linde Gas & Equipment Inc.,REQNOXBOX-I,330.0,02/08/2024,1142632930,2024


In [64]:
print(gen_df['covered_recipient_npi'].isnull().sum())
print(res_df['covered_recipient_npi'].isnull().sum())

21525
728842


In [65]:
gen_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 649560 entries, 0 to 15397507
Data columns (total 13 columns):
 #   Column                                                         Non-Null Count   Dtype  
---  ------                                                         --------------   -----  
 0   covered_recipient_profile_id                                   629267 non-null  float64
 1   covered_recipient_npi                                          628035 non-null  float64
 2   covered_recipient_first_name                                   629262 non-null  object 
 3   covered_recipient_last_name                                    629253 non-null  object 
 4   recipient_city                                                 649560 non-null  object 
 5   recipient_state                                                649514 non-null  object 
 6   applicable_manufacturer_or_applicable_gpo_making_payment_id    649560 non-null  int64  
 7   applicable_manufacturer_or_applicable_gpo_making_p

In [ ]:
# DROP NA IF IMPORTANT VALUES MISSING, EX. LINK TO PHYSICIANS
subset = [
    "covered_recipient_npi",
    "total_amount_of_payment_usdollars",
    "date_of_payment",
]

print(len(gen_df))
gen_df = gen_df.dropna(subset=subset)
print(len(gen_df))

print(len(res_df))
res_df = res_df.dropna(subset=subset)
print(len(res_df))

649560
628035
756193
27351


In [67]:
# DATA TYPES AND CLEANING 
gen_df["covered_recipient_profile_id"] = gen_df["covered_recipient_profile_id"].astype("Int64")
gen_df["covered_recipient_npi"] = gen_df["covered_recipient_npi"].astype("Int64")

gen_df["covered_recipient_first_name"] = gen_df["covered_recipient_first_name"].str.strip()
gen_df["covered_recipient_last_name"] = gen_df["covered_recipient_last_name"].str.strip()

gen_df["recipient_city"] = gen_df["recipient_city"].str.strip().str.title()
gen_df["recipient_state"] = gen_df["recipient_state"].str.strip().str.upper()

gen_df["applicable_manufacturer_or_applicable_gpo_making_payment_id"] = gen_df["applicable_manufacturer_or_applicable_gpo_making_payment_id"].astype("Int64")
gen_df["applicable_manufacturer_or_applicable_gpo_making_payment_name"] = gen_df["applicable_manufacturer_or_applicable_gpo_making_payment_name"].str.strip()

gen_df["date_of_payment"] = pd.to_datetime(gen_df["date_of_payment"], format="%m/%d/%Y", errors="coerce")

gen_df["record_id"] = gen_df["record_id"].astype("Int64")
gen_df["program_year"] = gen_df["program_year"].astype("Int64")

res_df["covered_recipient_profile_id"] = res_df["covered_recipient_profile_id"].astype("Int64")
res_df["covered_recipient_npi"] = res_df["covered_recipient_npi"].astype("Int64")

res_df["covered_recipient_first_name"] = res_df["covered_recipient_first_name"].str.strip()
res_df["covered_recipient_last_name"] = res_df["covered_recipient_last_name"].str.strip()

res_df["recipient_city"] = res_df["recipient_city"].str.strip().str.title()
res_df["recipient_state"] = res_df["recipient_state"].str.strip().str.upper()

res_df["applicable_manufacturer_or_applicable_gpo_making_payment_id"] = res_df["applicable_manufacturer_or_applicable_gpo_making_payment_id"].astype("Int64")
res_df["applicable_manufacturer_or_applicable_gpo_making_payment_name"] = res_df["applicable_manufacturer_or_applicable_gpo_making_payment_name"].str.strip()

res_df["date_of_payment"] = pd.to_datetime(res_df["date_of_payment"], format="%m/%d/%Y", errors="coerce")

res_df["record_id"] = res_df["record_id"].astype("Int64")
res_df["program_year"] = res_df["program_year"].astype("Int64")

In [68]:
# ADD TRANSACTION FLAG
gen_df["transaction_type"] = "general"
res_df["transaction_type"] = "research"

In [69]:
gen_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 628035 entries, 11 to 15397486
Data columns (total 14 columns):
 #   Column                                                         Non-Null Count   Dtype         
---  ------                                                         --------------   -----         
 0   covered_recipient_profile_id                                   628035 non-null  Int64         
 1   covered_recipient_npi                                          628035 non-null  Int64         
 2   covered_recipient_first_name                                   628030 non-null  object        
 3   covered_recipient_last_name                                    628022 non-null  object        
 4   recipient_city                                                 628035 non-null  object        
 5   recipient_state                                                627993 non-null  object        
 6   applicable_manufacturer_or_applicable_gpo_making_payment_id    628035 non-null  Int64 

In [70]:
gen_df.head(10)

,covered_recipient_profile_id,covered_recipient_npi,covered_recipient_first_name,covered_recipient_last_name,recipient_city,recipient_state,applicable_manufacturer_or_applicable_gpo_making_payment_id,applicable_manufacturer_or_applicable_gpo_making_payment_name,name_of_drug_or_biological_or_device_or_medical_supply_1,total_amount_of_payment_usdollars,date_of_payment,record_id,program_year,transaction_type
11,263830,1821102930,DONALD,GRIFFITH,Houston,TX,100000000186,Mission Pharmacal Company,LITHOSTAT,19223.51,2024-08-22,1063488454,2024,general
12,263830,1821102930,DONALD,GRIFFITH,Houston,TX,100000000186,Mission Pharmacal Company,LITHOSTAT,16127.18,2024-11-27,1063488487,2024,general
13,307418,1760577639,DONALD,LAMM,Paradise Valley,AZ,100000000186,Mission Pharmacal Company,NaN,1188.29,2024-08-22,1063488488,2024,general
15,307418,1760577639,DONALD,LAMM,Paradise Valley,AZ,100000000186,Mission Pharmacal Company,NaN,1317.91,2024-11-27,1063488498,2024,general
337,337143,1427041557,MICHAEL,SCHWARTZ,Plano,TX,100000401809,Tigon Medical LLC,NaN,925.00,2024-04-30,1067494116,2024,general
339,337143,1427041557,MICHAEL,SCHWARTZ,Plano,TX,100000401809,Tigon Medical LLC,NaN,250.00,2024-07-24,1067494119,2024,general
342,154556,1417981580,KEITH,WATSON,Fort Worth,TX,100000401809,Tigon Medical LLC,NaN,1575.00,2024-04-30,1067494124,2024,general
395,984959,1821257130,DAVID,KUMMERFELD,Tyler,TX,100000401809,Tigon Medical LLC,NaN,1600.00,2024-07-24,1067523761,2024,general
485,380039,1609061837,DAVID,ANDORSKY,Boulder,CO,100000005526,Ipsen Bioscience Inc.,NaN,1230.00,2024-09-20,1067524300,2024,general
489,263830,1821102930,DONALD,GRIFFITH,Houston,TX,100000000186,Mission Pharmacal Company,LITHOSTAT,15856.41,2024-02-29,1063864556,2024,general


In [71]:
column_order = [
        "covered_recipient_profile_id",
        "covered_recipient_npi",
        "covered_recipient_first_name",
        "covered_recipient_last_name",
        "recipient_city",
        "recipient_state",
        "applicable_manufacturer_or_applicable_gpo_making_payment_id",
        "applicable_manufacturer_or_applicable_gpo_making_payment_name",
        "name_of_drug_or_biological_or_device_or_medical_supply_1",
        "total_amount_of_payment_usdollars",
        "date_of_payment",
        "record_id",
        "program_year",
        "transaction_type"
]

gen_df = gen_df[column_order]
res_df = res_df[column_order]
print(len(gen_df))
print(len(res_df))

628035
27351


In [72]:
unified_df = pd.concat([gen_df, res_df], ignore_index=True)
display(unified_df.head(10))

unified_df.info()

,covered_recipient_profile_id,covered_recipient_npi,covered_recipient_first_name,covered_recipient_last_name,recipient_city,recipient_state,applicable_manufacturer_or_applicable_gpo_making_payment_id,applicable_manufacturer_or_applicable_gpo_making_payment_name,name_of_drug_or_biological_or_device_or_medical_supply_1,total_amount_of_payment_usdollars,date_of_payment,record_id,program_year,transaction_type
0,263830,1821102930,DONALD,GRIFFITH,Houston,TX,100000000186,Mission Pharmacal Company,LITHOSTAT,19223.51,2024-08-22,1063488454,2024,general
1,263830,1821102930,DONALD,GRIFFITH,Houston,TX,100000000186,Mission Pharmacal Company,LITHOSTAT,16127.18,2024-11-27,1063488487,2024,general
2,307418,1760577639,DONALD,LAMM,Paradise Valley,AZ,100000000186,Mission Pharmacal Company,NaN,1188.29,2024-08-22,1063488488,2024,general
3,307418,1760577639,DONALD,LAMM,Paradise Valley,AZ,100000000186,Mission Pharmacal Company,NaN,1317.91,2024-11-27,1063488498,2024,general
4,337143,1427041557,MICHAEL,SCHWARTZ,Plano,TX,100000401809,Tigon Medical LLC,NaN,925.00,2024-04-30,1067494116,2024,general
5,337143,1427041557,MICHAEL,SCHWARTZ,Plano,TX,100000401809,Tigon Medical LLC,NaN,250.00,2024-07-24,1067494119,2024,general
6,154556,1417981580,KEITH,WATSON,Fort Worth,TX,100000401809,Tigon Medical LLC,NaN,1575.00,2024-04-30,1067494124,2024,general
7,984959,1821257130,DAVID,KUMMERFELD,Tyler,TX,100000401809,Tigon Medical LLC,NaN,1600.00,2024-07-24,1067523761,2024,general
8,380039,1609061837,DAVID,ANDORSKY,Boulder,CO,100000005526,Ipsen Bioscience Inc.,NaN,1230.00,2024-09-20,1067524300,2024,general
9,263830,1821102930,DONALD,GRIFFITH,Houston,TX,100000000186,Mission Pharmacal Company,LITHOSTAT,15856.41,2024-02-29,1063864556,2024,general


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 655386 entries, 0 to 655385
Data columns (total 14 columns):
 #   Column                                                         Non-Null Count   Dtype         
---  ------                                                         --------------   -----         
 0   covered_recipient_profile_id                                   655386 non-null  Int64         
 1   covered_recipient_npi                                          655386 non-null  Int64         
 2   covered_recipient_first_name                                   655381 non-null  object        
 3   covered_recipient_last_name                                    655373 non-null  object        
 4   recipient_city                                                 655386 non-null  object        
 5   recipient_state                                                655343 non-null  object        
 6   applicable_manufacturer_or_applicable_gpo_making_payment_id    655386 non-null  Int6

In [73]:
print(len(unified_df))
no_dupes = unified_df.drop_duplicates()
print(len(no_dupes))

655386
655386


In [74]:
export_path = Path("../exports/cms_unified_2024.csv")
#export_path.parent.mkdir(parents=True, exist_ok=True)
#unified_df.to_csv(export_path, index=False, encoding="utf-8")

In [75]:
unified_df.sample(10)

,covered_recipient_profile_id,covered_recipient_npi,covered_recipient_first_name,covered_recipient_last_name,recipient_city,recipient_state,applicable_manufacturer_or_applicable_gpo_making_payment_id,applicable_manufacturer_or_applicable_gpo_making_payment_name,name_of_drug_or_biological_or_device_or_medical_supply_1,total_amount_of_payment_usdollars,date_of_payment,record_id,program_year,transaction_type
226233,271171,1962601047,MOHAMMADBAGHER,ZIARI,Corona,CA,100000136416,"Regeneron Healthcare Solutions, Inc.",LIBTAYO,3136.00,2024-10-24,1114262157,2024,general
133329,27964,1619023223,Syed,Mustafa,Rochester,NY,100000000146,AstraZeneca Pharmaceuticals LP,FASENRA,2150.00,2024-06-28,1104462594,2024,general
364435,193455,1275624660,RAJVEER,SACHDEV,Garnerville,NY,100000000066,"Lilly USA, LLC",MOUNJARO,750.00,2024-01-25,1135330749,2024,general
401880,23618,1528008588,R,RASMUSSEN,Provo,UT,100000005384,"INTUITIVE SURGICAL, INC.",Da Vinci Surgical System,2000.00,2024-02-14,1137239443,2024,general
367952,258371,1689867434,JONATHON,CHAPMAN,Baton Rouge,LA,100000000066,"Lilly USA, LLC",OMVOH,2500.00,2024-06-10,1135080936,2024,general
401349,8176691,1821400839,JAMIE,TUNG,Palo Alto,CA,100000005384,"INTUITIVE SURGICAL, INC.",DAVINCI XI,1000.00,2024-06-12,1137226449,2024,general
456824,11344113,1033567409,SEYED,ABEDI,Cedar Park,TX,100000000263,Incyte Corporation,NaN,74.99,2024-12-04,1141273020,2024,general
603354,1022999,1164643094,AMY,GIMLEN,Manhattan Beach,CA,100000005613,"Align Technology, Inc.",Invisalign,75.00,2024-02-20,1153235006,2024,general
431835,11076366,1538792833,JENNIFER,MCBAIN,Albany,NY,100000000137,Astellas Pharma US Inc,NaN,84.99,2024-04-22,1139508129,2024,general
167641,71913,1578858973,ALIREZA,ASHRAFI,Middleboro,MA,100000005689,Straumann USA LLC,NaN,33.24,2024-04-08,1106601716,2024,general
